In [18]:
#Importing libraries

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS
import docx

In [19]:
doc = docx.Document('C:\\Users\\Raimundo\\Videos\\Fala por entrevistado word.docx')
text = ""

for paragraph in doc.paragraphs:
    text += paragraph.text + '\n'

print(text[1])


*


In [26]:
# Stopwords em português:

import nltk
#nltk.download('stopwords')

# Get NLTK's Portuguese stopwords
nltk_stopwords = nltk.corpus.stopwords.words('portuguese')

# Define your custom stopwords
custom_stopwords = [
    "sim", "ter", "pode", "acredito", "sempre", "falta", "parte", "toda", "muita"
]

# Combine NLTK's stopwords and your custom stopwords
stopwords = nltk_stopwords + custom_stopwords


In [27]:



# Define custom Portuguese stopwords
#portuguese_stopwords = set([
#    "a", "o", "e", "é", "à", "á", "às", "àquele", "do", "que", "não", "de", "uma", "um", "em",
#    "sim", "para", "seu", "cmo", "se", "ou", "ser", "no", "muito", "com", "as", "os", "da", "qual"])

# Use the predefined Portuguese stopwords set
portuguese_stopwords = STOPWORDS

# Create the Word Cloud with custom stopwords
word_cloud = WordCloud(
    collocations=False,
    background_color='white',
    width=800,
    height=400,
    stopwords=stopwords  # Use your custom stopwords set
).generate(text)

# Save the Word Cloud image
word_cloud.to_file('word_cloud.png')

print(word_cloud)